In [1]:
import gtsam
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X

from gtsam.examples import SFMdata
from gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)
from gtsam.utils import plot
%config Completer.use_jedi = False

In [2]:
R.from_rotvec([0,np.pi/4,0]).as_matrix()

array([[ 0.70710678,  0.        ,  0.70710678],
       [ 0.        ,  1.        ,  0.        ],
       [-0.70710678,  0.        ,  0.70710678]])

In [5]:
# in our example, we assume we are given many general camera images,
# each consisting of tag image locations, each of which lies in one of 
# four real cameras. The general camera origin is at a fixed pose relative to reeal 
# camera 0 which places its origin at the center of the four cameras, and its x axis
# forward on the robot, its y axis to the left on the robot, and its z axis up. 

# The factor graph can be used to solve for the landmarks, the between 
# factors from real cameras1-3 to the generalized camera (which is in fixed relation to
# camera 0), and the poses of the generalized camera. Which variables are adjusted can
# controlled by prior factors on those variables. Use BearingFactor3D to define the
# tag locations in the real camera images (assume we have camera internal calibration).
# test code

# Create ground truth tag locations general camera locations, and camera 0-3 locations

# Create array CT to store the four real camera poses. CT[i,:,:] is the transform from camera
# i to the generalized camera frame. The generalized camera origin is at the center of the four
# cameras with x forward, y to the left, and z up (relative to the robot). The real camera 
# origins are on the corners of a 60cm x 10cm (in y and x of the general camera) rectangle 
# centered on the general camera origin and in the general camera z=0 plane. The real cameras
# are numbered 0..3 counter-clockwise from the one at (.05, -.3) (distance units are in meters).
# The real camera x-z planes are coincident with the general camera x-y plane, and the real
# cameras are oriented to point outward at 45 degrees off of the x-y axes of the general camera.
CT = np.zeros((4, 4, 4))
CT[0, 0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0,3*np.pi/4,0])).as_matrix()
CT[0, :,3] = [ 0.05, -0.3, 0, 1]
CT[1,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0,np.pi/4,0])).as_matrix()
CT[1, :,3] = [ 0.05,  0.3, 0, 1]
CT[2,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0,-np.pi/4,0])).as_matrix()
CT[2, :,3] = [-0.05,  0.3, 0, 1]
CT[3,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0,-3*np.pi/4,0])).as_matrix()
CT[3, :,3] = [-0.05, -0.3, 0, 1]

# print(np.round(CT, decimals=4))

# The april tags are at the 20 locations given in the 4x20 array A (in homogeneous coordinates)
A = np.array([10*[3] + 10*[-3],
              4*list(range(0,5)),
              2*(5*[0] + 5*[1]),
              20*[1]])

# The generalized camera starts out at the world cooridnate frame and moves along the world y axis
T = np.zeros((5, 4,4))
T[0, 0:3,0:3] = np.identity(3)
T[0, :,3] = [0, 0, 0, 1]
T[1, 0:3,0:3] = np.identity(3)
T[1, :,3] = [0, 1, 0, 1]
T[2, 0:3,0:3] = np.identity(3)
T[2, :,3] = [0, 2, 0, 1]
T[3, 0:3,0:3] = R.from_rotvec([0,0,np.pi/6]).as_matrix()
T[3, :,3] = [0, 2, 1, 1]
T[4, 0:3,0:3] = R.from_rotvec([0,0,np.pi/4]).as_matrix()
T[4, :,3] = [1, 2, 0, 1]

# Project features to produce P and O. Assume tags project into cameras 0-3 based on their x-y 
# quadrant in the generalized camera frame
def project(T, A):
    """
       T is the transform from the generalized camera to the world
       A is a 4 by n array with the homogeneous coordinates of the tags
       
       Returns:
       O: n by 3 array of the projection ray origins in the generalized camera frame
       P: n by 3 array of the projection ray directions (to the tags) in the generalized camera frame
    """
    signs2camera= [[2, 1], [3, 0]]
    n = A.shape[1]
    G = np.linalg.inv(T) @ A   # tags in generalized camera frame
    O = np.zeros((n,3))
    P = np.zeros((n,3))
    for p in range(n):
        camera = signs2camera[int(G[0,p] >= 0)][int(G[1,p] >= 0)]
        O[p,:] = CT[camera, 0:3, 3]
        # transform the tag in generalized camera coords to the real camera coords, then
        # rotate this direction back to generalized camera coords again and normalize to unit length
        P[p,:] = CT[camera, 0:3,0:3] @ (np.linalg.inv(CT[camera, :,:]) @ G[:, p])[0:3]
        P[p,:] /= np.linalg.norm(P[p,:])
        
    return P, O



[[[-0.7071  0.      0.7071  0.05  ]
  [-0.7071  0.     -0.7071 -0.3   ]
  [-0.     -1.     -0.      0.    ]
  [ 0.      0.      0.      1.    ]]

 [[ 0.7071  0.      0.7071  0.05  ]
  [-0.7071  0.      0.7071  0.3   ]
  [-0.     -1.      0.      0.    ]
  [ 0.      0.      0.      1.    ]]

 [[ 0.7071  0.     -0.7071 -0.05  ]
  [ 0.7071  0.      0.7071  0.3   ]
  [ 0.     -1.      0.      0.    ]
  [ 0.      0.      0.      1.    ]]

 [[-0.7071  0.     -0.7071 -0.05  ]
  [ 0.7071  0.     -0.7071 -0.3   ]
  [ 0.     -1.     -0.      0.    ]
  [ 0.      0.      0.      1.    ]]]


In [ ]:

# Add noise to ground truth for initialziation of optimization

# project GT into tag image locations and add noise

# run bundle adjust to estimate optimal values

In [ ]:
# create factor graph
g = g = gtsam.NonlinearFactorGraph
